Loading the data

In [1]:
from sklearn.datasets import load_digits

digits = load_digits()

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

Extract necessary data

In [2]:
import numpy as np

X = []
y = []
for i in range(target.shape[0]):
    if target[i]==3:
        X.append(data[i])
        y.append(1)
    elif target[i]==8:
        X.append(data[i])
        y.append(-1)
    else: continue

y = np.array(y)
X = np.array(X)
ones = np.ones(X.shape[0],dtype=float)
ones = ones[:, np.newaxis]
X = np.concatenate((X, ones),axis=1)
print("Shape of X is:", X.shape)
print("Shape of y is:", y.shape)

Shape of X is: (357, 65)
Shape of y is: (357,)


Logistic Regression and Cross Validation

In [14]:
import sklearn as sk
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv = 100, Cs = 100, random_state=0, 
                        solver ="liblinear",
                        fit_intercept = False,
                        refit = True,
                        multi_class="ovr").fit(X,y)

print("The best value for C is:", clf.C_)
print("The score is:", clf.score)
print("The regularization parameter is:", 1/clf.C_)
print(clf.Cs)

The best value for C is: [0.00413201]
The score is: <bound method LogisticRegressionCV.score of LogisticRegressionCV(Cs=100, class_weight=None, cv=100, dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=None, penalty='l2', random_state=0,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)>
The regularization parameter is: [242.01282648]
100


1.2 Optimization Methods

In [16]:
def sigmoid(z):
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            z[i][j] = 1/(1+np.exp(-z))
    return z

def gradient(beta, X,y):